<a href="https://colab.research.google.com/github/ZAHID188/Q-learning_collab/blob/main/Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
print('Hi')

Hi


In [2]:
class deeepQNetwork(object):
  #learning rate =lr, number of action=n_actions,name= name of actions
  #fcl_dim= fully connected layer dimension,
  def __init__(self,lr,n_actions,name,fcl_dims=256,input_dims=(210,160,4),chkpt_dir='tmp/dqn'):
    self.lr=lr
    self.name=name
    self.n_actions=n_actions
    self.fcl_dims=fcl_dims
    self.input_dims=input_dims
    self.sess=tf.Session()
    self.build_network()
